In [1]:
%load_ext autoreload
%autoreload 2

import ase.io
import numpy as np

import rascaline
import equistore
from equistore import Labels, TensorBlock, TensorMap
import chemiscope

# from rascaline.utils import clebsch_gordan
import clebsch_gordan
import old_clebsch_gordan
import rotations

In [2]:
frames = ase.io.read("combined_magres_spherical.xyz", ":")

# Define hyperparameters for generating the rascaline SphericalExpansion
rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

# chemiscope.show(frames, mode="structure")

# Equivariance Test - SO(3) for $\nu=3$

In [3]:
# Define target lambda channels
lambdas = np.array([0, 2])

# Generate Wigner-D matrices, initialized with random angles
wig = rotations.WignerDReal(lmax=rascal_hypers["max_angular"])
print("Random rotation angles (rad):", wig.angles)

# Randomly rigidly rotate the frame
frames_so3 = [rotations.transform_frame_so3(frame, wig.angles) for frame in frames]
assert not np.allclose(frames[-1].positions, frames_so3[-1].positions)

# Generate nu=3 descriptor for both frames
nu3 = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    only_keep_parity=+1,
)

nu3_rot = clebsch_gordan.n_body_iteration_single_center(
    frames_so3,
    rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    only_keep_parity=+1,
)

# Rotate the lambda-SOAP descriptor of the unrotated frame
nu3_transformed = wig.transform_tensormap_so3(nu3)

# Check for equivariance!
assert equistore.equal_metadata(nu3_transformed, nu3_rot)
assert equistore.allclose(nu3_transformed, nu3_rot)
print("SO(3) EQUIVARIANT!")

# chemiscope.show([frame, frame_so3], mode="structure")

Random rotation angles (rad): [5.94258177 1.33221078 0.11693219]
SO(3) EQUIVARIANT!


# Equivariance Test - O(3) on $\nu=2$ (i.e. $\lambda$-SOAP)

In [4]:
# Define target lambda channels
lambdas = np.array([0, 1, 2, 3, 4, 5])

# Generate Wigner-D matrices, initialized with random angles
wig = rotations.WignerDReal(lmax=rascal_hypers["max_angular"])
print("Random rotation angles (rad):", wig.angles)

# Apply an O(3) transformation to the frame
frames_o3 = [rotations.transform_frame_o3(frame, wig.angles) for frame in frames]
assert not np.allclose(frames[-1].positions, frames_o3[-1].positions)

# Generate lambda-SOAP for both frames
lsoap = clebsch_gordan.lambda_soap_vector(
    frames,
    rascal_hypers,
    lambdas=lambdas,
    only_keep_parity=+1,
)

lsoap_o3 = clebsch_gordan.lambda_soap_vector(
    frames_o3,
    rascal_hypers,
    lambdas=lambdas,
    only_keep_parity=+1,
)

# Apply the O(3) transformation to the TensorMap
lsoap_transformed = wig.transform_tensormap_o3(lsoap)

# Check for equivariance!
assert equistore.equal_metadata(lsoap_transformed, lsoap_o3)
assert equistore.allclose(lsoap_transformed, lsoap_o3)
print("O(3) EQUIVARIANT!")

# chemiscope.show([frame, frame_o3], mode="structure")

Random rotation angles (rad): [2.07830924 0.730489   5.29996543]
O(3) EQUIVARIANT!


# Old v new lambda-SOAP

In [ ]:
# Define target lambda channels
lambdas = np.arange(6)
lambdas

In [ ]:
lsoap_old = old_clebsch_gordan.lambda_soap_vector(
    frames,
    rascal_hypers,
    lambdas=lambdas,
    lambda_cut=8,
)
lsoap_old

In [ ]:
lsoap_new = clebsch_gordan.lambda_soap_vector(
    frames,
    rascal_hypers,
    lambdas=lambdas,
    lambda_cut=8,
    only_keep_parity=+1,
)
lsoap_new

# Dense v Sparse

In [ ]:
# Dense
lsoap_new0 = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
    lambda_cut=5,
    use_sparse=False,
)
lsoap_new0

In [ ]:
# Sparse
lsoap_new1 = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
    lambda_cut=5,
    use_sparse=True,
    only_keep_parity=+1,
)
lsoap_new1

In [ ]:
# Check sparse == dense
equistore.allclose(lsoap_new0, lsoap_new1)

# Test system 1

In [ ]:
%%timeit

use_sparse = False

tensor_dense = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=use_sparse,
)
tensor_dense

# timeit comes out at about 22 seconds for nu_target = 3

In [ ]:
%%timeit

use_sparse = True

tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=use_sparse,
)
tensor_sparse

# timeit comes out at about 13 seconds for nu_target = 3

In [ ]:
tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=True,
)

tensor_dense = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=False,
)

assert equistore.allclose(tensor_dense, tensor_sparse)

# Test system 2

In [ ]:
frames = [ase.io.read("frame.xyz")]

lambdas = np.array([0, 2])

rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

n_body = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
)
n_body

In [ ]:
def sort_tm(tm):
    blocks = []
    for _, block in tm.items():
        values = block.values

        samples_values = block.samples.values
        sorted_idx = native_list_argsort([tuple(row.tolist()) for row in block.samples.values])
        samples_values = samples_values[sorted_idx]
        values = values[sorted_idx]

        components_values = []
        for i, component in enumerate(block.components):
            component_values = component.values
            sorted_idx = native_list_argsort([tuple(row.tolist()) for row in component.values])
            components_values.append( component_values[sorted_idx] )
            values = np.take(values, sorted_idx, axis=i+1)

        properties_values = block.properties.values
        sorted_idx = native_list_argsort([tuple(row.tolist()) for row in block.properties.values])
        properties_values = properties_values[sorted_idx]
        values = values[..., sorted_idx]

        blocks.append(
            TensorBlock(
                values=values,
                samples=Labels(values=samples_values, names=block.samples.names),
                components=[Labels(values=components_values[i], names=component.names) for i, component in enumerate(block.components)],
                properties=Labels(values=properties_values, names=block.properties.names)
            )
        )
    return TensorMap(keys=tm.keys, blocks=blocks)


def native_list_argsort(native_list):
    return sorted(range(len(native_list)), key=native_list.__getitem__)


In [ ]:

# Manipulate metadata from old LSOAP
lsoap_old = equistore.permute_dimensions(lsoap_old0, axis="properties", dimensions_indexes=[2, 5, 0, 1, 3, 4])
lsoap_old = equistore.rename_dimension(lsoap_old, axis="properties", old="l_1", new="l1")
lsoap_old = equistore.rename_dimension(lsoap_old, axis="properties", old="l_2", new="l2")
lsoap_old = equistore.rename_dimension(lsoap_old, axis="properties", old="n_1", new="n1")
lsoap_old = equistore.rename_dimension(lsoap_old, axis="properties", old="n_2", new="n2")

# Slice TM to symmetrize l-values
sliced_blocks = []
for key, block in lsoap_old.items():
    # Filter properties l1 <= l2
    mask = [entry[0] <= entry[1] for entry in block.properties]
    new_labels = Labels(names=block.properties.names, values=block.properties.values[mask])

    # Slice block
    sliced_block = equistore.slice_block(block, axis="properties", labels=new_labels)
    sliced_blocks.append(sliced_block)

# Check equal metadata
lsoap_old = sort_tm(TensorMap(keys=lsoap_old.keys, blocks=sliced_blocks))
lsoap_new = sort_tm(lsoap_new0)
assert equistore.equal_metadata(lsoap_old, lsoap_new)

# Check equal values
equistore.allclose_raise(lsoap_old, lsoap_new)

In [ ]:
for key in lsoap_old.keys:
    b1 = lsoap_old[key]
    b2 = lsoap_new[key]

    print(key, equistore.allclose_block(b1, b2))

In [ ]:
np.linalg.norm(
    lsoap_old.block(inversion_sigma=1, spherical_harmonics_l=1, species_center=3).values
    - lsoap_new.block(
        inversion_sigma=1, spherical_harmonics_l=1, species_center=3
    ).values
)

In [ ]:
# NOW SLICE TO l1 == l2

# Slice TM
sliced_blocks = []
for key, block in lsoap_old.items():
    # Filter properties
    mask = [entry[0] == entry[1] for entry in block.properties]
    new_labels = Labels(names=block.properties.names, values=block.properties.values[mask])

    # Slice block
    sliced_blocks.append(equistore.slice_block(block, axis="properties", labels=new_labels))

lsoap_old_sliced = TensorMap(keys=lsoap_old.keys, blocks=sliced_blocks)

# Slice TM
sliced_blocks = []
for key, block in lsoap_new.items():
    # Filter properties
    mask = [entry[0] == entry[1] for entry in block.properties]
    new_labels = Labels(names=block.properties.names, values=block.properties.values[mask])

    # Slice block
    sliced_blocks.append(equistore.slice_block(block, axis="properties", labels=new_labels))

lsoap_new_sliced = TensorMap(keys=lsoap_new.keys, blocks=sliced_blocks)

assert equistore.equal_metadata(lsoap_old_sliced, lsoap_new_sliced)
assert equistore.allclose_raise(lsoap_old_sliced, lsoap_new_sliced)